# Notebook 2: Vertical Slice Gravity wave

This notebook will demonstrate using Gusto to solve the compressible Euler equations in a 2D (x-z) domain, often called a 'vertical slice'. The variables in this system are the velocity $\textbf{u}$, the dry density $\rho$, and the (virtual dry) potential temperature $\theta$. The fluid is stratified and the background state is in hydrostatic balance. The initial conditions consist of a perturbation to $\theta$ in the centre of the domain. This setup is given in the 1994 paper by Skamarock and Klemp: https://journals.ametsoc.org/view/journals/mwre/122/11/1520-0493_1994_122_2623_eaaotk_2_0_co_2.xml.

As our standard first step, we begin by importing the required libraries and functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import (as_vector, SpatialCoordinate, PeriodicIntervalMesh,
                       ExtrudedMesh, exp, sin, pi, Function)

ModuleNotFoundError: No module named 'gusto'

Specify the time-step size, simulation duration, and frequency of output results.

In [2]:
dt = 6.
tmax = 3600.
dumpfreq = 300    # this will output every 300 timesteps

For this vertical slice model, we first define a periodic interval mesh  with width equal to that of the required domain and the number of cells equal to the number of columns we require in our final mesh. The periodic mesh is then extruded in the vertical direction by specifying the number of layers and the layer height. We now have a two dimensional mesh with vertically aligned columns.

In [3]:
nlayers = 10  # number of horizontal layers
columns = 150  # number of columns
domain_width = 3.0e5
m = PeriodicIntervalMesh(columns, domain_width)

domain_height = 1.0e4  # Height position of the model top
mesh = ExtrudedMesh(m, layers=nlayers, layer_height=domain_height/nlayers)

NameError: name 'PeriodicIntervalMesh' is not defined

As always, we initialise our `state` that evolves with each time-step. The finite element family specified in the `CompressibleEulerEquations` class corresponds to the velocity space of the horizontal mesh. As this is a 1D mesh, Continuous Galerkin (CG) elements are used. As Gusto solves directly on the vertical slice domain, none of the fields are in the 2D CG space.

In [4]:
domain = Domain(mesh, dt, "CG", 1)
parameters = CompressibleParameters()
eqns = CompressibleEulerEquations(domain, parameters)

NameError: name 'Domain' is not defined

We now set up the `OutputParameters` class, specifying the path to the output directory `dirname` and passing in `dumpfreq`. The default behaviour is for the vtu output files to contain all of the prognostic fields. In this case we would like to visualise the evolution of the perturbation, so we specify those as diagnostic fields, along with the potential temperature gradient and the Richardson number.

In [5]:
dirname = 'gravity_wave'
output = OutputParameters(dirname=dirname,
                          dumpfreq=dumpfreq)
Tsurf = 300.
diagnostic_fields = [
        CourantNumber(), Gradient('u'), Perturbation('theta'),
        Gradient('theta_perturbation'), Perturbation('rho'),
        RichardsonNumber('theta', parameters.g/Tsurf), Gradient('theta')
    ]
io = IO(domain, output, diagnostic_fields=diagnostic_fields)

NameError: name 'OutputParameters' is not defined

In [6]:
theta_opts = SUPGOptions()
subcycling_options = SubcyclingOptions(subcycle_by_courant=0.25)
transported_fields = [
    SSPRK3(domain, "u", subcycling_options=subcycling_options),
    SSPRK3(
        domain, "rho", subcycling_options=subcycling_options,
        rk_formulation=RungeKuttaFormulation.linear
    ),
    SSPRK3(
        domain, "theta", subcycling_options=subcycling_options,
        options=theta_opts
    )
]
transport_methods = [
    DGUpwind(eqns, "u"),
    DGUpwind(eqns, "rho", advective_then_flux=True),
    DGUpwind(eqns, "theta", ibp=theta_opts.ibp)
]

NameError: name 'SUPGOptions' is not defined

Set up the initial conditions and define the thermodynamic constants.

In [7]:
# Initial conditions
u0 = stepper.fields("u")
rho0 = stepper.fields("rho")
theta0 = stepper.fields("theta")

# spaces
Vt = state.spaces("theta")
Vr = state.spaces("DG")

# Thermodynamic constants required for setting initial conditions
# and reference profiles
g = parameters.g
N = parameters.N
p_0 = parameters.p_0
c_p = parameters.cp
R_d = parameters.R_d
kappa = parameters.kappa

xz = SpatialCoordinate(mesh)

# N^2 = (g/theta)dtheta/dz => dtheta/dz = theta N^2g => theta=theta_0exp(N^2gz)
thetab = Tsurf*exp(N**2*z/g)

theta_b = Function(Vt).interpolate(thetab)
rho_b = Function(Vr)

NameError: name 'stepper' is not defined

This next step takes the temperature profile and computes the corresponding density that will enforce a hydrostatic balance.

In [8]:
compressible_hydrostatic_balance(state, theta_b, rho_b)

NameError: name 'compressible_hydrostatic_balance' is not defined

We now set up the initial potential temperature profile. We apply a perturbation $\theta'$ to the background balanced $\theta_b$ defined above, where $\theta'$ is given by:
$$\theta' = \Delta \theta \frac{\sin\big(\frac{\pi z}{H}\big)}{1 + \frac{1}{a^2}(x-x_c)^2}$$

In [9]:
a = 5.0e3
deltaTheta = 1.0e-2
theta_pert = deltaTheta*sin(pi*xz[1]/H)/(1 + (xz[0] - L/2)**2/a**2)
theta0.interpolate(theta_b + theta_pert)
rho0.assign(rho_b)
u0.project(as_vector([20.0, 0.0]))

stepper.set_reference_profiles([('rho', rho_b),
                              ('theta', theta_b)])

NameError: name 'sin' is not defined

A streamline upwind Petrov-Galerkin (SUPG) transport scheme will be applied to the potential temperature.

Set up the linear solver and the time-stepper. As per the first notebook, we use an Implicit Midpoint method for the velocity field, and an explicit SSPRK3 scheme for the other variables (density and potential temperature). This time, we need to specify the use of a compressible solver.

Next, you would solve the equations! However, they take ages to run - do we run for a few time steps? The only problem is that it will create output which will overwrite the current results.

In [10]:
# Here's what you would do...
# stepper.run(t=0, tmax=tmax)

To save you having to run this full simulation, results have already been generated and stored in the 'results/gravity_wave' subdirectory. We will show a visualisation of the potential temperature at the points we specified earlier. 

We set-up the visualisations that can be used in Jupyter notebook: